In [4]:
import pickle
def load_clients(pickle_path):
    try:
        with open(pickle_path, "rb") as f:
            clients = pickle.load(f)
        return clients
    except Exception as e:
        print(f"Failed to load pickle file: {e}")
        return None

full_data = load_clients("clients.pkl")

In [61]:
import phonenumbers
import pycountry
import re

def get_country_code(country_name: str) -> str:
    try:
        return pycountry.countries.lookup(country_name.strip()).alpha_2
    except LookupError:
        return None

def normalize_phone_number(phone_number: str, region_code: str) -> str:
    # Replace starting '00' with '+'
    phone_number = phone_number.strip()
    if phone_number.startswith("00"):
        phone_number = "+" + phone_number[2:]

    # Remove all spaces, dashes, etc., but keep leading +
    cleaned = re.sub(r"[^\d+]", "", phone_number)

    # If it starts with '+', check and remove extra '0' after country code
    if cleaned.startswith("+"):
        try:
            parsed = phonenumbers.parse(cleaned, None)
            actual_cc = str(parsed.country_code)
            if cleaned.startswith(f"+{actual_cc}0"):
                # Remove the 0 after country code
                cleaned = f"+{actual_cc}{cleaned[len(actual_cc)+2:]}"
                
        except:
            pass
    else:
        # Local number without country code — assume local and prepend +country_code
        if not cleaned.startswith("0"):
            cleaned = "0" + cleaned
        try:
            country_calling_code = phonenumbers.country_code_for_region(region_code)
            cleaned = f"+{country_calling_code}{cleaned.lstrip('0')}"
        except:
            return phone_number  # fallback
    return cleaned

def is_dutch_mobile(number: str) -> bool:
    number = re.sub(r"[^\d]", "", number)
    return number.startswith("316") and len(number) == 11

def check_phone_number_with_country_name(phone_number: str, country_names: str) -> bool:
    country_list = re.split(r"[;,:\|]+", country_names)

    for name in country_list:
        region_code = get_country_code(name)
        if not region_code:
            continue
        try:
            normalized_number = normalize_phone_number(phone_number, region_code)
            parsed = phonenumbers.parse(normalized_number, None)
            is_valid = phonenumbers.is_valid_number(parsed)
            region = phonenumbers.region_code_for_number(parsed)
            if (normalized_number.startswith("316") and len(normalized_number) == 11 or len(normalized_number) == 10 or normalized_number.startswith("+316") and len(normalized_number) == 12) and region_code=="NL":
                is_valid = True
            if is_valid and region == region_code:
                return True
        except phonenumbers.NumberParseException:
            continue
    return False

# Examples
print(check_phone_number_with_country_name("+45 53 11 20 42", "Denmark"))         # True
print(check_phone_number_with_country_name("53 11 20 42", "Denmark"))             # True
print(check_phone_number_with_country_name("+49 53 11 20 42", "Denmark"))         # False
print(check_phone_number_with_country_name("+33 6 12 34 56 78", "Denmark,France"))# True
print(check_phone_number_with_country_name("+43 660 1234567", "Germany; Austria"))# True
print(check_phone_number_with_country_name("+310676538069", "Netherlands"))  # True (fixes +310 → +316)
print(check_phone_number_with_country_name("0676538069", "Netherlands"))     # True (adds +31)
print(check_phone_number_with_country_name("0043676538069", "Austria"))      # True (fixes 00 to +)
print(check_phone_number_with_country_name("06 76 538 069", "Netherlands"))  # True (cleans format)




True
True
False
True
True
True
True
True
True


In [42]:
print(check_phone_number_with_country_name("+31676538069", "Netherlands"))  # True (fixes +310 → +316)

NL
False
Country Code: 31 National Number: 676538069
False


In [43]:
import phonenumbers

number = "+31676538069"

try:
    parsed = phonenumbers.parse(number, None)
    print("Parsed:", parsed)
    print("Valid:", phonenumbers.is_valid_number(parsed))
    print("Region:", phonenumbers.region_code_for_number(parsed))
except phonenumbers.NumberParseException as e:
    print("Error:", e)

Parsed: Country Code: 31 National Number: 676538069
Valid: False
Region: NL


In [44]:
import phonenumbers

num = phonenumbers.parse("+31676538069", None)
print("Valid:", phonenumbers.is_valid_number(num))  # Should now be True


Valid: False


True


In [59]:
client_labels = []
for clients in full_data:
    if clients['label']['label'] == 'Reject':
        client_labels.append(False)
    elif clients['label']['label'] == 'Accept':
        client_labels.append(True)

In [62]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
flags_preds = []       
for client in full_data:
    flags = True
    if not check_phone_number_with_country_name(client['client_profile']['phone_number'], client['client_profile']['country_of_domicile']):
        flags = False
    flags_preds.append(flags)



In [64]:
for i in range (0, len(full_data)):
    #Just need to check if the client phone number with country is getting rejected but the equivalent flag is True
    if check_client_labels(full_data[i], flags_preds[i]) == 0 and client_labels[i] == True:
        print("Phone Number: ", full_data[i]['client_profile']['phone_number'])
        print("Country of Domicile: ", full_data[i]['client_profile']['country_of_domicile'])
        print("Label: ", full_data[i]['label']['label'])
        print("\n")
        print("----------------------------------------------------------")


false negative
Phone Number:  +41 003 320 03 27
Country of Domicile:  Switzerland
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  12 96 25 74
Country of Domicile:  Denmark
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  +32 0430 333 666
Country of Domicile:  Belgium
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  0424 724 765
Country of Domicile:  Belgium
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  +41 098 284 92 04
Country of Domicile:  Switzerland
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  +43 491 177 6094
Country of Domicile:  Austria
Label:  Accept


----------------------------------------------------------
false negative
Phone Number:  +45 02 02 64 69
Country of Domicile:  Denmark
Label

In [23]:
check_phone_number_with_country_name("+310676538069", "Netherlands")

False

In [63]:
from sklearn.metrics import confusion_matrix
import numpy as np
array_flags = np.array(flags_preds)
array_labels = np.array(client_labels)
confusion_matrix(np.array(flags_preds), np.array(client_labels))

array([[1506, 1302],
       [3486, 3706]], dtype=int64)

In [9]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
        
for client in full_data:
    flags = True
    if client['passport']['last_name'] != client['account_form']['last_name']:
        flags = False
    check_client_labels(client, flags)

In [75]:
from datetime import datetime

from datetime import datetime

def is_high_school_graduation_year_valid(graduation_year: int, birth_date: str, min_age: int = 16, max_age: int = 21) -> bool:
    """
    Check if the graduation year is reasonable given a person's birth date.

    Args:
        graduation_year (int): The year the person graduated high school.
        birth_date (str): Birth date in "YYYY-MM-DD" format.
        min_age (int): Minimum expected age at graduation (default: 16)
        max_age (int): Maximum expected age at graduation (default: 23)

    Returns:
        bool: True if the graduation year is within a reasonable age range.
    """
    try:
        birth_year = datetime.strptime(birth_date, "%Y-%m-%d").year
        age_at_graduation = graduation_year - birth_year
        return min_age <= age_at_graduation <= max_age
    except ValueError:
        print("Invalid birth date format. Use YYYY-MM-DD.")
        return False





In [76]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
flags_preds = []       
for client in full_data:
    flags = True
    if not is_high_school_graduation_year_valid(client['client_profile']['secondary_school']['graduation_year'], client['client_profile']['birth_date']):
        flags = False
    flags_preds.append(flags)

from sklearn.metrics import confusion_matrix
import numpy as np
array_flags = np.array(flags_preds)
array_labels = np.array(client_labels)
confusion_matrix(np.array(flags_preds), np.array(client_labels))

array([[ 736,    0],
       [4256, 5008]], dtype=int64)

In [10]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
        
for client in full_data:
    flags = True
    if client['passport']['first_name'] != client['account_form']['first_name']:
        flags = False
    check_client_labels(client, flags)

In [11]:
full_data[0].keys()

dict_keys(['passport', 'client_profile', 'account_form', 'client_description', 'label'])

In [ ]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
        
for client in full_data:
    flags = True
    if client['passport']['first_name'] != client['account_form']['first_name']:
        flags = False
    check_client_labels(client, flags)

In [77]:
from countryinfo import CountryInfo
import pycountry

def check_multiple_country_consistency(codes_str, countries_str, nationalities_str):
    # Split input strings by commas and strip whitespace
    codes = [c.strip() for c in codes_str.split(',')]
    countries = [c.strip() for c in countries_str.split(',')]
    nationalities = [n.strip() for n in nationalities_str.split(',')]

    # Ensure all lists have the same length
    if not (len(codes) == len(countries) == len(nationalities)):
        return False, "Input lists must be of the same length"

    results = []

    for i, (code, country_name, nationality) in enumerate(zip(codes, countries, nationalities)):
        try:
            # 1. Validate country code
            country = pycountry.countries.get(alpha_2=code.upper())
            if not country:
                results.append((False, f"[{i}] Invalid country code: {code}"))
                continue

            # 2. Check if country name matches code
            if country.name.lower() != country_name.lower():
                results.append((False, f"[{i}] Mismatch: code '{code}' is for '{country.name}', not '{country_name}'"))
                continue

            # 3. Check nationality using CountryInfo
            info = CountryInfo(country.name)
            expected_nationality = info.info().get("demonym", None)

            if not expected_nationality:
                results.append((False, f"[{i}] Could not find nationality for '{country.name}'"))
                continue

            if nationality.lower() != expected_nationality.lower():
                results.append((False, f"[{i}] Nationality '{nationality}' does not match expected '{expected_nationality}' for {country.name}"))
                continue

            # All good for this triplet
            results.append((True, f"[{i}] All fields are consistent and valid"))

        except KeyError:
            results.append((False, f"[{i}] CountryInfo does not recognize '{country_name}'"))
        except Exception as e:
            results.append((False, f"[{i}] Error: {str(e)}"))

    return results


In [78]:
def check_client_labels(clients, flags):
    if clients['label']['label'] == 'Reject' and flags == False:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == True:
        return 1
    elif clients['label']['label'] == 'Accept' and flags == False:
        print("false negative")
        return 0
    return 0
flags_preds = []       
for client in full_data:
    flags = True
    if not check_multiple_country_consistency(client["passport"]["country_code"],client["passport"]["country"],client["passport"]["nationality"]):
        flags = False
    flags_preds.append(flags)

from sklearn.metrics import confusion_matrix
import numpy as np
array_flags = np.array(flags_preds)
array_labels = np.array(client_labels)
confusion_matrix(np.array(flags_preds), np.array(client_labels))

array([[   0,    0],
       [4992, 5008]], dtype=int64)